# 1. 文档

案例大全：https://www.windquant.com/qntcloud/apiTemplateMenu/id-d1d00878-8e85-40d3-bf91-bde9401f0b8a

帮助文档：https://www.windquant.com/qntcloud/apiRefHelp/id-91573a98-70d5-4462-8c6f-546ab45c8652

注意事项：
* 每次限8000单元格
* 数据流量的计算方法是滚动计算，从当前时间前推7*24小时，这段时间内的用量超过了限制，
  则需要过一段时间再用，等到按前面所说计算方法计算出来的用量不再超过限制的流量。 
  具体每个函数的流量限制，请联系客户经理咨询

In [1]:
# 1. 连接万得服务
from WindPy import w

w.start() # 默认命令超时时间为120秒，如需设置超时时间可以加入waitTime参数，例如waitTime=60,即设置命令超时时间为60秒  
w.isconnected() # 判断WindPy是否已经登录成功
# w.stop() # 当需要停止WindPy时，可以使用该命令
          # 注： w.start不重复启动，若需要改变参数，如超时时间，用户可以使用w.stop命令先停止后再启动。
          # 退出时，会自动执行w.stop()，一般用户并不需要执行w.stop 

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


True

# 2. 获取股票列表

对应接口“获取报表数据函数WSET”

In [2]:
# wset函数支持输出DataFrame数据格式，需要函数添加参数usedf = True, 如例2。可以使用usedfdt=True来填充DataFrame输出NaT的日期
# sectorconstituent：板块成分

# 获取A股所有列表
all_a = w.wset("SectorConstituent",date = '2022-09-20' ,sector=u"全部A股", usedf = True)
all_a[1].head(5)

,date,wind_code,sec_name
1,2022-09-20,000001.SZ,平安银行
2,2022-09-20,000002.SZ,万科A
3,2022-09-20,000004.SZ,ST国华
4,2022-09-20,000005.SZ,ST星源
5,2022-09-20,000006.SZ,深振业A


In [7]:
# 获取沪深300所有列表
stock_codes = w.wset("sectorconstituent", "date=2018-06-12;windcode=000300.SH;field=wind_code,sec_name", usedf = True)
stock_codes[1].head(5)

,wind_code,sec_name
1,000001.SZ,平安银行
2,000002.SZ,万科A
3,000060.SZ,中金岭南
4,000063.SZ,中兴通讯
5,000069.SZ,华侨城A


In [5]:
# 获取申万一级行业的成分股
sw_index=w.wset("sectorconstituent","date=2018-06-12;sectorid=a39901011g000000;field=wind_code,sec_name",usedf=True)
sw_index[1].head(5)

,wind_code,sec_name
1,801010.SI,农林牧渔(申万)
2,801030.SI,基础化工(申万)
3,801040.SI,钢铁(申万)
4,801050.SI,有色金属(申万)
5,801080.SI,电子(申万)


In [13]:
# 获取所有停牌股票
st_codes = w.wset("tradesuspend","startdate=2022-08-22;enddate=2022-09-22;",usedf=True)
st_codes[1].head(5)

,date,wind_code,sec_name,suspend_type,suspend_reason
1,2022-08-22,000670.SZ,盈方微,盘中停牌,盘中成交价较开盘价首次上涨达到或超过10%，盘中成交价较开盘价首次上涨达到或超过20%
2,2022-08-22,000972.SZ,ST中基,停牌一天,重大事项
3,2022-08-22,300018.SZ,中元股份,今起停牌,重大事项
4,2022-08-22,300419.SZ,浩丰科技,今起停牌,重大事项
5,2022-08-22,300521.SZ,爱司凯,停牌一天,重大事项


# 3. 板块信息获取

In [6]:
# 成分行业分布: 仅有11个
# 输出字段：code: 代码，name：行业名称，count: 成份个数，marketcap:总市值，
#   freesharesmv:自由流通市值，weight: 权重(%)，point: 贡献点
sec_industry = w.wset("constituentsindustrydist","windcode=000300.SH;enddate=2022-09-22;industry=Wind一级行业",usedf=True)
sec_industry[1].head(5)

,code,name,count,marketcap,freesharesmv,weight,point
1,882007.WI,金融,49,130707.910205,36800.309687,20.1791,-3.691025
2,882011.WI,房地产,6,6692.101775,3522.772276,1.8438,-1.295100
3,882008.WI,信息技术,61,51289.954607,27329.989327,16.1443,-4.020317
4,882004.WI,可选消费,25,34377.825323,13477.991229,8.1729,-5.773307
5,882003.WI,工业,47,58939.112800,25509.596964,15.4582,-4.002825


In [9]:
# 这些ID只能在代码生成器中去查，没有接口获取ID和对应中文的列表
# 提取上证A股和深证A股的当日平均收盘价信息。
errorCode,data=w.wsee("a001010200000000,a001010300000000","sec_close_avg","tradeDate=20180827",usedf=True)
data.head(5)

,SEC_CLOSE_AVG
a001010200000000,13.520050
a001010300000000,13.270491


# 4. 获取行情数据
## 4.1 历史日行情数据查询

In [2]:
# !!!!!! 每次限8000单元格!!!!
# 数据流量的计算方法是滚动计算，从当前时间前推7*24小时，
# 这段时间内的用量超过了限制，则需要过一段时间再用，等到按前面所说计算方法计算出来的用量不再超过限制的流量。 
# 具体每个函数的流量限制，请联系客户经理咨询

# 查询浦发银行，后复权，没有数据填充空值
# PriceAdj: 复权，前复权-F，后复权-B，无复权则不写这个参数
# 查询字段：
# 指标名称    字段
# 开盘价      open
# 最高价      high
# 最低价      low
# 收盘价      close
# 成交量      volume
# 涨跌        chg
# 成交额      amt
# 均价        vwap
# 前收盘价    pre_close
# 复权因子    adjfactor    
# 换手率      turn
# 交易状态    trade_status  
# 停牌原因    susp_reason
# 涨跌停状态  maxupordown
# 涨停价      maxup
# 跌停价      maxdown
# 开盘集合竞价成交价   open_auction_price   
# 开盘集合竞价成交量   open_auction_volume  股（张）
# 开盘集合竞价成交额   open_auction_amount  元

data_market = w.wsd("600000.SH", "open,high,low,close,volume,chg,amt,vwap,pre_close,adjfactor,"
                   "turn,trade_status,susp_reason,maxupordown,maxup,maxdown,open_auction_price,"
                   "open_auction_volume,open_auction_amount", 
      "2022-08-22", "2022-09-22", "unit=1;PriceAdj=B",usedf=True)
data_market[1].head(5)

,OPEN,HIGH,LOW,CLOSE,VOLUME,CHG,AMT,VWAP,PRE_CLOSE,ADJFACTOR,TURN,TRADE_STATUS,SUSP_REASON,MAXUPORDOWN,MAXUP,MAXDOWN,OPEN_AUCTION_PRICE,OPEN_AUCTION_VOLUME,OPEN_AUCTION_AMOUNT
2022-08-22,106.728977,107.620862,106.431682,107.472215,13776616.0,0.03,99353492.0,107.200913,107.026272,14.86476,0.046936,交易,None,0,7.92,6.48,7.18,422253.0,3031776.54
2022-08-23,107.174920,107.323567,106.580329,106.580329,15047471.0,-0.06,108111340.0,106.798619,107.472215,14.86476,0.051265,交易,None,0,7.95,6.51,7.21,60300.0,434763.00
2022-08-24,106.580329,107.323567,106.134386,106.283034,16855765.0,-0.02,120996053.0,106.703985,106.580329,14.86476,0.057426,交易,None,0,7.89,6.45,7.17,71877.0,515358.09
2022-08-25,106.431682,107.323567,106.283034,107.026272,15772556.0,0.05,113287117.0,106.766830,106.283034,14.86476,0.053736,交易,None,0,7.87,6.44,7.16,57900.0,414564.00
2022-08-26,107.472215,107.918158,106.728977,107.323567,17576235.0,0.02,126876441.0,107.303290,107.026272,14.86476,0.059881,交易,None,0,7.92,6.48,7.23,187100.0,1352733.00


## 4.2 获取日大盘指数

In [2]:
# 获取上证指数000001.SH，收盘价，前收盘价
errorCode, data=w.wss("000001.SH", "close,pre_close","tradeDate=20220923;priceAdj=U;cycle=D", usedf=True)
data.head(5)

,CLOSE,PRE_CLOSE
000001.SH,3088.3695,3108.9096


# 5. 获取交易日列表

In [7]:
errorCode, data = w.tdays("2022-08-28", "2022-09-28", "", usedf=True)
data.head(5)

,
2022-08-29,2022-08-29
2022-08-30,2022-08-30
2022-08-31,2022-08-31
2022-09-01,2022-09-01
2022-09-02,2022-09-02
